# Regression

PreFace:
Dear Professor Zhang and TA Mr.Yin:
In xgboost, it is not only time-consuming but also impossible to check the ratio of completion while finetuning. Due to these two inconveniences, I do not use gridsearchcv. Instead, I choose to test each parameter in each value separately, so that I can better control the process.

In [1]:
import pandas as pd
import numpy as np
import sys 
import statsmodels as sm
import sklearn
import scipy as sp
%matplotlib inline 
import matplotlib.pyplot as plt
import math
import warnings;
warnings.filterwarnings('ignore')
import xgboost as xgb
from sklearn.model_selection import train_test_split as tr_te_split
from scipy.stats.mstats import winsorize
from sklearn.linear_model import LinearRegression as LinReg

# (a)Data Pre-processing

In [2]:
data=pd.read_csv('dataframe_train.csv')
data_test_official=pd.read_csv('dataframe_test.csv')
#先处理好categorical数据,split train-test,最后再处理outliers.

In [3]:
df = data.drop(columns=['courier_id','aoi_id','shop_id','date','tracking_id','action_type','group','id'])
df=pd.get_dummies(df,columns=['source_type','weather_grade'])
df_train=df[0:484136]
df_test=df[484136:]

Dealing with outliers in ['urgency']

In [4]:
Q3=df_train.urgency.quantile(0.75)
Q1=df_train.urgency.quantile(0.25)
IQR=Q3-Q1
lower_limit=Q1-1.5*IQR
upper_limit=Q3+1.5*IQR

In [5]:
X_train=df_train[(df_train['urgency']>=lower_limit)&(df_train['urgency']<=upper_limit)].drop(columns=['expected_use_time'])
y_train=df_train[(df_train['urgency']>=lower_limit)&(df_train['urgency']<=upper_limit)]['expected_use_time'].ravel()
X_test=df_test.drop(columns=['expected_use_time'])
y_test=df_test['expected_use_time'].ravel()

In [6]:
def mae(y_test,y_pred):
    mae=sum(abs(y_test-y_pred)/len(y_pred))
    return mae

# Baseline model: Linear Regression

In [7]:
reg=LinReg(fit_intercept=True).fit(X_train,y_train)
y_pred=reg.predict(X_test)

In [8]:
mae(y_test,y_pred)

219.82118156063947

# Try polynomial

In [22]:
from sklearn.preprocessing import PolynomialFeatures

In [23]:
poly=PolynomialFeatures(degree=2)
X_train_poly, X_test_poly=poly.fit_transform(X_train),poly.fit_transform(X_test)
model=LinReg()
model=model.fit(X_train_poly,y_train)

In [24]:
y_pred_1=model.predict(X_test_poly)
mae(y_test, y_pred_1)

219.82295709014724

# Tree Regression

In [25]:
from sklearn.tree import DecisionTreeRegressor
Courier_tree = DecisionTreeRegressor(ccp_alpha=0.005,max_depth=4)
Courier_tree.fit(X_train,y_train)

DecisionTreeRegressor(ccp_alpha=0.005, max_depth=4)

In [26]:
y_pred=Courier_tree.predict(X_test)

In [27]:
mae(y_test, y_pred)

196.2748716191463

# xg-boost

In [9]:
data_train = xgb.DMatrix(data=X_train,label= y_train)
data_test = xgb.DMatrix(data=X_test,label= y_test)

In [92]:
xgb_reg = xgb.XGBRegressor(objective ='reg:squarederror',
                           colsample_bynode=0.9,\
                           learning_rate = 0.1,
                           gamma = 0.5,max_depth = 15, 
                           n_estimators = 17)
xgb_reg.fit(X_train,y_train)
preds = xgb_reg.predict(X_test)

Parameters' benchmark: colsample_bynode=0.8 learning_rate=0.1, gamma=0.001, max_depth=20, n_estimators = 200.

Testing max_depth (range:[10,12,13,14,16,20]):

In [11]:
mae(y_test,preds) #20

188.1209758820999

In [13]:
#开始调参:由于gridsearchcv把5个参数都代入会带来巨大的时间损耗并且过程不可控,所以我把5个参数分开来,分别进行调参.
mae(y_test,preds) #max_depth=15 n_estimators=200

182.6459564596142

In [15]:
mae(y_test,preds) #10

183.17397181023492

In [17]:
mae(y_test,preds) #13

182.74315823085217

In [19]:
mae(y_test,preds)#12

183.31424015868393

In [21]:
mae(y_test,preds) #14

182.85617851827863

In [23]:
mae(y_test,preds) #16

184.57764357628304

The optimal max_depth = 15. 

Now we test the n_estimators (in range[10,300])

In [27]:
mae(y_test,preds) #300

182.67674987061633

In [29]:
mae(y_test,preds) #100

183.23533971617482

In [31]:
mae(y_test,preds) #50

184.56003399815492

In [33]:
mae(y_test,preds) #40

183.83339741143143

In [35]:
mae(y_test,preds) #30

181.65527008691325

In [37]:
mae(y_test,preds) #20

177.20331352440886

In [39]:
mae(y_test,preds) #15

177.29318678041668

In [41]:
mae(y_test,preds) #10

177.29318678041668

In [43]:
mae(y_test,preds) #21

177.57841713047216

In [45]:
mae(y_test,preds) #19

176.83233290687858

In [47]:
mae(y_test,preds) #18

176.6111852395695

In [49]:
mae(y_test,preds) #max_depth =15 n_estimators=17 learning_rate=0.1

176.52909411199965

In [54]:
mae(y_test,preds) #16

176.6829544607436

In [56]:
mae(y_test,preds) #15

177.29318678041668

The optimal n_estimators is 17

Now we test the learning_rate [0.005, 0.01, 0.05, 0.1, 0.3]:

In [71]:
mae(y_test,preds) #max_depth =15 n_estimators=17 learning_rate=0.1

176.52909411199965

In [58]:
mae(y_test,preds) #=0.05

202.8258841597758

In [60]:
mae(y_test,preds) #=0.01

338.6743368963736

In [62]:
mae(y_test,preds) #=0.005

367.736894730359

In [69]:
mae(y_test,preds) #=0.3

188.99043374270272

The optimal learning rate is 0.1

Test colsample_bynode [0.6,0.7,0.8,0.9]:

In [73]:
mae(y_test,preds) #max_depth =15 n_estimators=17 learning_rate=0.1 colsample_bynode=0.8

176.52909411199965

In [75]:
mae(y_test,preds) #0.6

177.61557123957684

In [77]:
mae(y_test,preds) #0.7

177.12254687733247

In [79]:
mae(y_test,preds) #max_depth =15 n_estimators=17 learning_rate=0.1 colsample_bynode=0.9

176.47737277149426

The optimal colsample_bynode=0.9

Testing gamma=[0.001,0.005,0.01,0.05,0.1]:

In [83]:
mae(y_test,preds) #0.001

176.47737277149426

In [85]:
mae(y_test,preds) #0.005

176.47737277149426

In [87]:
mae(y_test,preds) #0.01

176.47737266705744

In [89]:
mae(y_test,preds) #0.05

176.4773736319093

In [91]:
mae(y_test,preds) #0.1

176.477372400479

In [93]:
mae(y_test,preds) #0.5

176.47733619240358

Now, the gamma almost do not change the final mae. Therefore, the local optimal gamma is 0.5.

The optimal parameters are max_depth =15 n_estimators=17 learning_rate=0.1 colsample_bynode=0.9 gamma=0.5.

Final submission.

In [7]:
df1=data.drop(columns=['courier_id','aoi_id','shop_id','date','tracking_id','action_type','group','id'])
df1=pd.get_dummies(df1,columns=['source_type','weather_grade'])
Q3=df_train.urgency.quantile(0.75)
Q1=df_train.urgency.quantile(0.25)
IQR=Q3-Q1
lower_limit=Q1-1.5*IQR
upper_limit=Q3+1.5*IQR
X_train_final=df1[(df1['urgency']>=lower_limit)&(df1['urgency']<=upper_limit)].drop(columns=['expected_use_time'])
y_train_final=df1[(df1['urgency']>=lower_limit)&(df1['urgency']<=upper_limit)]['expected_use_time'].ravel()
X_test_final=data_test_official.drop(columns=['courier_id','aoi_id','shop_id','date','tracking_id','group','id'])
X_test_final=pd.get_dummies(X_test_final,columns=['source_type','weather_grade'])

In [8]:
data_train = xgb.DMatrix(data=X_train_final,label= y_train_final)
xgb_reg = xgb.XGBRegressor(objective ='reg:squarederror',
                           colsample_bynode=0.9,\
                           learning_rate = 0.1,
                           gamma = 0.5,
                           max_depth = 15,
                           n_estimators = 17)
xgb_reg.fit(X_train_final,y_train_final)
preds = xgb_reg.predict(X_test_final)

In [9]:
df_classification = pd.read_csv("regression_example.csv",index_col = 0)
df_classification["expected_use_time"] = preds
df_classification.to_csv("regression_xgboost_final.csv")

MAE is 181.66473.

# (d) Feature Engineering

Here I would try polynomialfeatures.

In-sample

In [12]:
from sklearn.preprocessing import PolynomialFeatures
poly=PolynomialFeatures(degree=2)
X_train_poly, X_test_poly=poly.fit_transform(X_train),poly.fit_transform(X_test)
model=LinReg()
model=model.fit(X_train_poly,y_train)
y_pred_1=model.predict(X_test_poly)

In [13]:
data_train = xgb.DMatrix(data=X_train_poly,label= y_train)
data_test = xgb.DMatrix(data=X_test_poly,label= y_test)

In [71]:
xgb_reg = xgb.XGBRegressor(objective ='reg:squarederror',
                           colsample_bynode=0.9,\
                           learning_rate = 0.1,
                           gamma = 1,max_depth = 15, 
                           n_estimators = 18)
xgb_reg.fit(X_train_poly,y_train)
preds = xgb_reg.predict(X_test_poly)

Finetune

test max_depth

In [16]:
mae(y_test, preds)# max_depth=15 n_estimators=17

177.0093791205657

In [21]:
mae(y_test, preds)# 20

180.02005714046612

In [23]:
mae(y_test, preds)# 10

178.00025857043443

In [25]:
mae(y_test, preds)# 14

177.27639994223233

In [27]:
mae(y_test, preds)# 16

177.57710131130648

optimal max_depth is 15

let's test n_estimators

In [29]:
mae(y_test, preds)# max_depth=15 n_estimators=50

186.16153905459663

In [31]:
mae(y_test, preds)# max_depth=15 n_estimators=100

184.36105692818634

In [33]:
mae(y_test, preds)# max_depth=15 n_estimators=18 colsample_bynode=0.9 learning rate=0.1

176.9852020311301

In [35]:
mae(y_test, preds)# max_depth=15 n_estimators=16

177.284882616215

In [37]:
mae(y_test, preds)# max_depth=15 n_estimators=19

177.15317453404037

In [39]:
mae(y_test, preds)# max_depth=15 n_estimators=20

177.6156524540927

So the optimal n_estimators is 18.

Let's test the colsample_bynode

In [41]:
mae(y_test, preds)# 0.8

177.67838378943583

In [43]:
mae(y_test, preds)# 1.0

177.855954532543

The local optimal colsample_bynode is 0.9.

test learning rate

In [64]:
mae(y_test, preds) #learning rate=0.1 gamma=0.5

176.9852020311301

In [56]:
mae(y_test, preds) #0.05

199.67043346142222

In [59]:
mae(y_test, preds) #0.01

335.69827049321873

In [61]:
mae(y_test, preds) #0.3

188.90448711162497

So the optimal learning_rate is 0.1

Let's test the gamma.

In [66]:
mae(y_test, preds)# 0.1

177.2964411210768

In [68]:
mae(y_test, preds)# 0.01

177.2078117829236

In [70]:
mae(y_test, preds)# 1

176.9488770102186

In [72]:
mae(y_test, preds)# 2

177.22701946951702

The optimal gamma is 1.

Now test the. out-of-sample.

In [76]:
from sklearn.preprocessing import PolynomialFeatures
poly=PolynomialFeatures(degree=2)
X_train_final_poly, X_test_final_poly=poly.fit_transform(X_train_final),poly.fit_transform(X_test_final)
model=LinReg()
model=model.fit(X_train_final_poly,y_train_final)
y_pred_1=model.predict(X_test_final_poly)

In [79]:
data_train = xgb.DMatrix(data=X_train_final_poly,label= y_train_final)

In [81]:
xgb_reg = xgb.XGBRegressor(objective ='reg:squarederror',
                           colsample_bynode=0.9,\
                           learning_rate = 0.1,
                           gamma = 1,max_depth = 15, 
                           n_estimators = 18)
xgb_reg.fit(X_train_final_poly,y_train_final)
preds_final_poly = xgb_reg.predict(X_test_final_poly)

In [82]:
df_classification = pd.read_csv("regression_example.csv",index_col = 0)
df_classification["expected_use_time"] = preds_final_poly
df_classification.to_csv("regression_xgboost_final——poly.csv")

Out-of-sample MAE is 181.72844.

# (e)Model interpretation

In [10]:
importance=pd.DataFrame(list(xgb_reg.get_booster().get_score(importance_type='gain').items()),columns=['features','importance']).sort_values(by='importance',ascending=False)
importance

,features,importance
0,source_type_ASSIGN,3.153250e+09
19,source_type_PICKUP,6.740365e+06
1,grid_distance,4.530666e+06
2,source_type_DELIVERY,4.330676e+06
3,urgency,1.370349e+06
9,weather_grade_Very Bad Weather,9.970780e+05
17,source_tracking_id,6.235317e+05
16,target_lat,5.576708e+05
14,target_lng,5.176822e+05
12,hour,4.596205e+05


According to the importance, the top several most relevant features in predicting the expected_use_time are 'source_type', 'grid_distance', 'urgency','weather_grade'.